In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt

C:\Users\USER\anaconda3\envs\ai\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [3]:
Data = np.load('D:/Dropbox/AICleansing_ver2/2.ProcessedData/train_mimic_orgscale.npy')
# TrData = np.load('D:/Dropbox/AICleansing_ver2/2.ProcessedData/train_mimic_orgscale.npy')
# ValData = np.load('D:/Dropbox/AICleansing_ver2/2.ProcessedData/valid_vitaldb_orgscale.npy')

In [4]:
np.random.shuffle(Data)
Data_len = len(Data)
ValData = Data[:len(Data)//5]
TrData = Data[len(Data)//5:]
print(TrData.shape, ValData.shape)

(283380, 3000) (70844, 3000)


In [5]:
#Ver2

TrDataFrame = tf.signal.frame(TrData.astype('float32'), 1, 1).numpy()
ValDataFrame = tf.signal.frame(ValData.astype('float32'), 1, 1).numpy()

# np.random.shuffle(TrDataFrame)
# np.random.shuffle(ValDataFrame)

TrMask = np.random.choice([0,1], size=TrDataFrame.shape, p=[0.1,0.9])
ValMask = np.random.choice([0,1], size=ValDataFrame.shape, p=[0.1,0.9])

Tr_X = TrDataFrame.copy()
Tr_Y = (TrDataFrame - 20.0) / (220.0 - 20.0)
# 평균 80, 표준 편차 25인 정규 분포에서 값을 뽑아서 채울 배열 생성
random_values = np.random.normal(loc=80, scale=25, size=TrDataFrame.shape)
# TrMask가 1인 위치에 뽑은 값을 할당
Tr_X[TrMask == 0] = random_values[TrMask == 0]
Tr_X = (Tr_X - 20.0) / (220.0 - 20.0)

TrMask[:,-10:,:] = 0
Tr_X[:,-10:,:] = Tr_X[:,-10:,:] + np.random.normal(loc=0.0, scale=0.05, size=Tr_X[:,-10:,:].shape)

Tr_X = np.clip(Tr_X, 0.0, 1.0)

Val_X = ValDataFrame.copy()
Val_Y = (ValDataFrame - 20.0) / (220.0 - 20.0)
# 평균 80, 표준 편차 25인 정규 분포에서 값을 뽑아서 채울 배열 생성
random_values = np.random.normal(loc=80, scale=25, size=ValDataFrame.shape)
# TrMask가 1인 위치에 뽑은 값을 할당
Val_X[ValMask == 0] = random_values[ValMask == 0]
Val_X = (Val_X - 20.0) / (220.0 - 20.0)

ValMask[:,-10:,:] = 0
Val_X[:,-10:,:] = Val_X[:,-10:,:] + np.random.normal(loc=0.0, scale=0.05, size=Val_X[:,-10:,:].shape)

Val_X = np.clip(Val_X, 0.0, 1.0)

del random_values
del TrDataFrame
del ValDataFrame

In [6]:
Length = Tr_X.shape[1]
OrigDim = Tr_X.shape[-1]


NumLayers=2
EmbedDim = 64
NumHead = 1
DimFC = 200
BottleneckSize = EmbedDim // 8
Groups = NumHead * 2
HeadDim = EmbedDim // NumHead
NumLayers = 2

In [7]:
d_model = 500

In [8]:
class DilatedBottleneckBlock(tf.keras.layers.Layer):
    def __init__(self,  out_channel=64, bottleneck=BottleneckSize, kernel_size=15, dilation=1, groups=Groups, firstlayergroups=None):
        super(DilatedBottleneckBlock, self).__init__()
        self.firstlayergroups = firstlayergroups

        # Bottle
        self.bottle = tf.keras.layers.Conv1D(bottleneck, kernel_size=1, groups=groups)
        
        # ReLU
        self.relu = tf.keras.layers.ReLU()

        # BatchNorm
        self.bn = tf.keras.layers.BatchNormalization(center=True, scale=True)

        # Dilated Conv
        dilation_padding = (kernel_size - 1) // 2 * dilation
        if firstlayergroups:
            self.dilated_conv = tf.keras.layers.Conv1D(out_channel, kernel_size, dilation_rate=dilation, padding='same', groups=firstlayergroups)
        else:
            self.dilated_conv = tf.keras.layers.Conv1D(out_channel, kernel_size, dilation_rate=dilation, padding='same', groups=groups)

    def call(self, x):
        bottle_out = self.bottle(x)
        conv_out = self.dilated_conv(bottle_out)
        
        if self.firstlayergroups:
            return self.bn(self.relu(conv_out) + tf.tile(x, [1, 1, 2]))
        else:
            return self.bn(self.relu(conv_out) + x)

class DilatedBottleneckNet(tf.keras.Model):
    def __init__(self,  out_channel=256, bottleneck=BottleneckSize, kernel_size=15, dilation=50, groups=Groups):
        super(DilatedBottleneckNet, self).__init__()

        self.layer0 = DilatedBottleneckBlock( out_channel * 2, bottleneck * 2, kernel_size, dilation, 1, firstlayergroups=groups)
        self.layer1 = DilatedBottleneckBlock( out_channel * 2, bottleneck * 2, kernel_size, dilation * 2, groups)
        self.layer2 = DilatedBottleneckBlock( out_channel * 2, bottleneck * 2, kernel_size, dilation * 4, groups)
        self.layer3 = DilatedBottleneckBlock( out_channel * 2, bottleneck * 2, kernel_size, dilation * 8, groups)
        self.layer4 = DilatedBottleneckBlock( out_channel * 2, bottleneck * 2, kernel_size, dilation * 16, groups)
        self.layer5 = DilatedBottleneckBlock( out_channel * 2, bottleneck * 2, kernel_size, dilation * 32, groups)

    def call(self, x):
            x0 = self.layer0(x)
            x1 = self.layer1(x0)
            x2 = self.layer2(x1)
            x3 = self.layer3(x2)
            x4 = self.layer4(x3)
            x5 = self.layer5(x4)
            
            return x5 # return shape (batch, length, embed)

    

In [9]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)# (..., seq_len_q, seq_len_k)
    
    
    
    
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [10]:
class CustomAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, bottleneck=BottleneckSize, groups=Groups, rate=0.1, t_len=-1):
        super(CustomAttention, self).__init__()
    
        self.num_heads = num_heads
        self.d_model = d_model
        self.scaling = num_heads ** -0.5
        self.t_len = t_len 

        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads
        
        self.DBN = DilatedBottleneckNet(out_channel =d_model , bottleneck = bottleneck, groups=groups)
        
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        self.dr = tf.keras.layers.Dropout(rate)
        
        

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size,  self.t_len, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,  x, training, mask):
        
        
        
        q, k =  tf.split(self.DBN(x), 2, axis=-1)
        batch_size = tf.shape(q)[0]
        v = self.wv(x)

        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # Scaling q
        q = q * self.scaling

        # Computing the batch matrix multiplication
        AttOutWeights =  tf.linalg.matmul(q, k, transpose_b=True)
        
        # Masking before normalization for proper attention distribution
        Mask_repeated = tf.tile(mask[:, None], [1, self.num_heads, 1, 1])
        masked_out_weights = tf.multiply(AttOutWeights, Mask_repeated) 
        
        # Subtracting max value for numerical stability before taking exponential
        max_vals = tf.math.reduce_max(masked_out_weights, axis=-1, keepdims=True)
        AttOutputWeights = tf.exp(masked_out_weights - max_vals) 
        
        # Applying normalization
        sum_vals = tf.math.reduce_sum(AttOutputWeights, axis=-1, keepdims=True)
        AttOutputWeights /= sum_vals
        
        # Applying dropout
        AttOutputWeights = self.dr(AttOutputWeights)

        
        # Computing attention output using batched matrix multiplication
        AttOutput = tf.linalg.matmul(AttOutputWeights, v)

        # Reshaping and transposing tensors
        AttOutput = tf.reshape(tf.transpose(AttOutput, perm=[0, 2, 1, 3]), [-1, self.t_len, self.d_model])
        
        
        '''
        scaled_attention, attention_weights = scaled_dot_product_attention( q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = tf.reshape(scaled_attention,  (-1, self.t_len, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        '''
        
        return AttOutput

In [11]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])



class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, bottleneck=BottleneckSize, groups=Groups,  rate=0.1, t_len=-1):
        super(EncoderLayer, self).__init__()

        
                
        self.catt = CustomAttention(d_model, num_heads, bottleneck=bottleneck, t_len=t_len)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
        self.wv = tf.keras.layers.Dense(d_model)
        
        
    def call(self, x, training, mask):

        
        attn_output = self.catt(x, training, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
        
            
        return out2

In [12]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, bottleneck=BottleneckSize, groups=Groups,  rate=0.1, t_len=-1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.convembed = tf.keras.layers.Conv1D(filters=d_model, kernel_size=11, strides=1, padding='same', dilation_rate=1)
        self.bn = tf.keras.layers.BatchNormalization()
        
      
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate=rate, t_len=t_len)  for _ in range(num_layers)]
        

        #self.dropout = tf.keras.layers.Dropout(rate)

    
    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]
        
        x = self.convembed(x)
        x = self.bn(x)
        x = tf.nn.relu(x)

     
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        
        
        return x  # (batch_size, input_seq_len, d_model)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

# ModelCheckpoint 설정
checkpoint_filepath = './results/BDC_rightedge_mask_only_mimic.hdf5'  # 저장할 파일 경로 및 이름
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',  # 검증 데이터의 loss를 기준으로 함
    mode='min',  # loss의 최솟값을 모니터링
    save_best_only=True  # 가장 낮은 loss 값을 가질 때만 저장
)

In [14]:
InpLayer = tf.keras.layers.Input((Length, 1))
InpMask  = tf.keras.layers.Input((Length, 1))
EmbedOut = Encoder(num_layers = NumLayers, d_model = EmbedDim, num_heads = NumHead, dff = DimFC, t_len=Length)(InpLayer, True, InpMask)

# After Encoder
Projection = Conv1D(filters=OrigDim, kernel_size=11, strides=1, padding='same', dilation_rate=1)(EmbedOut)
#Projection= Activation('sigmoid')(Projection)

learning_rate = 0.00001  # replace this with your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

BDC = Model([InpLayer,InpMask ], Projection)
BDC.compile(loss='mse', optimizer = optimizer, )

In [18]:
BDC.fit([Tr_X, TrMask], Tr_Y, validation_data=([Val_X, ValMask], Val_Y), batch_size=30, verbose=1, shuffle=True, epochs=10000, callbacks=[model_checkpoint_callback] )

Epoch 1/10000
9446/9446 [==============================] - 2522s 267ms/step - loss: 0.0595 - val_loss: 0.0069
Epoch 2/10000
9446/9446 [==============================] - 2520s 267ms/step - loss: 0.0034 - val_loss: 0.0020
Epoch 3/10000
9446/9446 [==============================] - 2521s 267ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 4/10000
9446/9446 [==============================] - 2521s 267ms/step - loss: 0.0011 - val_loss: 9.6499e-04
Epoch 5/10000
9446/9446 [==============================] - 2521s 267ms/step - loss: 8.1492e-04 - val_loss: 6.9623e-04
Epoch 6/10000
9446/9446 [==============================] - 2521s 267ms/step - loss: 6.4386e-04 - val_loss: 5.5289e-04
Epoch 7/10000
9446/9446 [==============================] - 2521s 267ms/step - loss: 5.1484e-04 - val_loss: 4.4233e-04
Epoch 8/10000
9446/9446 [==============================] - 2521s 267ms/step - loss: 4.0957e-04 - val_loss: 3.5504e-04
Epoch 9/10000
9446/9446 [==============================] - 2523s 267ms/step - loss: 

KeyboardInterrupt: 

In [36]:
BottleneckSize, Groups

(32, 8)

In [12]:
def Trans_qkz (tensor, dims):
    tensor = tf.transpose(tensor, perm=[2, 0, 1])
    tensor = tf.reshape(tensor, dims)
    return tf.transpose(tensor, perm=[1, 0, 2])

In [15]:
BTModel = DilatedBottleneckNet(out_channel =EmbedDim , bottleneck = BottleneckSize, groups=Groups)
q, k =  tf.split(BTModel(Embed), 2, axis=-1)
v = tf.keras.layers.Dense(EmbedDim, use_bias=True)(Embed)

print(q)
print(k)
print(v)

q =  tf.transpose(q, (1,2,0)) # [batch size, channel, length]
k =  tf.transpose(k, (1,2,0)) # [batch size, channel, length] 
v =  tf.transpose(v, (1,0,2)) # [batch size, length, channel]



#input before the view() should be [len, bs, channel]
bsz = 100
q = Trans_qkz (q, [Length, bsz * NumHead, HeadDim])    
k = Trans_qkz (k, [Length, bsz * NumHead, HeadDim])    
v = Trans_qkz (v, [Length, bsz * NumHead, HeadDim])  
#output should be [bsz * num_heads, len, head_dim]

SrcLen = tf.shape(k)[1]

q,k, v, SrcLen

KerasTensor(type_spec=TensorSpec(shape=(None, 500, 128), dtype=tf.float32, name=None), name='tf.split_3/split:0', description="created by layer 'tf.split_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 500, 128), dtype=tf.float32, name=None), name='tf.split_3/split:1', description="created by layer 'tf.split_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 500, 128), dtype=tf.float32, name=None), name='dense_5/BiasAdd:0', description="created by layer 'dense_5'")


(<KerasTensor: shape=(400, 500, 32) dtype=float32 (created by layer 'tf.compat.v1.transpose_25')>,
 <KerasTensor: shape=(400, 500, 32) dtype=float32 (created by layer 'tf.compat.v1.transpose_27')>,
 <KerasTensor: shape=(400, 500, 32) dtype=float32 (created by layer 'tf.compat.v1.transpose_29')>,
 <KerasTensor: shape=() dtype=int32 inferred_value=[500] (created by layer 'tf.__operators__.getitem_2')>)

In [34]:
q

<KerasTensor: shape=(None, 500, 256) dtype=float32 (created by layer 'tf.split')>

In [20]:
sample_encoder_layer = EncoderLayer(512, 8, 2048)
sample_encoder_layer_output = sample_encoder_layer( tf.random.uniform((20, 43, 512)), False, None)
sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

TensorShape([20, 43, 512])

In [26]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, input_vocab_size=8500,
                         maximum_position_encoding=10000)
temp_input = tf.random.uniform((64, 62), dtype=tf.int64, minval=0, maxval=200)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

(64, 62, 512)


In [7]:
in_channel=256
out_channel=256
bottleneck=32
kernel_size=15
dilation=1
groups=2
DilatedBottleneckBlock(in_channel, out_channel * 2, bottleneck * 2, kernel_size, dilation, 1, firstlayergroups=groups)

NameError: name 'DilatedBottleneckBlock' is not defined

In [75]:
class ConvEmbedding(tf.keras.layers.Layer):
    def __init__(self, orig_dim=12, embed_dim=32, **kwargs):
        super(ConvEmbedding, self).__init__(**kwargs)

        self.embedding = tf.keras.Sequential([
            tf.keras.layers.Conv1D(filters=embed_dim, kernel_size=11, strides=1, padding='same', dilation_rate=1, input_shape=(None, orig_dim)),
            tf.keras.layers.BatchNormalization(axis=-1, trainable=True, center=True, scale=True, momentum=0.99, epsilon=0.001),
            tf.keras.layers.ReLU()
        ])

    def call(self, x):
        # Current shape (batch_size, length, channels)
        #x = tf.transpose(x, perm=[0, 2, 1])
        x1 = self.embedding(x)
        return x1

In [36]:
Bottleneck=32
kernel_size=15
dilation=1
groups=2

### Setting 

In [26]:
mimic_ppg_test = np.load('./mimic_ppg_test.npy')

# shape [batch_size, embed_dim, length]
Length = mimic_ppg_test.shape[1]
OrigDim = mimic_ppg_test.shape[-1]
EmbedDim = 100
NumHead = 4

In [10]:
def _get_clones(layer, N):
    return [tf.keras.models.clone_model(layer) for _ in range(N)]

In [27]:
InpLayer = Input(shape=(Length, OrigDim,  ))

## ConvEmbedding ##
Embed  =  Conv1D(filters=EmbedDim, kernel_size=11, strides=1, padding='same', dilation_rate=1, input_shape=(None, OrigDim))(InpLayer)
Embed = BatchNormalization(axis=-1, trainable=True, center=True, scale=True)(Embed)
Embed = tf.transpose(Activation('relu')(Embed) , (0,2,1)) # shape [batch_size, embed_dim, length]
## ConvEmbedding ##

Embed =  tf.transpose(Embed, (2,0,1)) # shape [length, batch_size, embed_dim]

In [19]:
#EmbedOrgin = tf.identity(Embed)
#Embed = tf.transpose(Embed, perm=[1, 0, 2])
#Embed = self.bottleneck(Embed)

#Embed = tf.transpose(Embed, perm=[2, 0, 1])

In [ ]:
output = self.self_attn(Embed, pos_embed, return_attn_weights)
q, k = self.q_k_conv(query.permute(1,2,0)).chunk(2, dim=1) # query should be [batch size, channel, length]
# q_k_conv -> dilated_bottleneck_net

In [34]:
TLen, BatSize, EmbedDim = Embed.shape.as_list()
HeadDim = EmbedDim // NumHead
Scaling = float(HeadDim) ** -0.5

Embed =  tf.transpose(Embed, perm=[1, 2, 0]) #query.permute(1,2,0)

In [35]:
Embed

<KerasTensor: shape=(None, 100, 30000) dtype=float32 (created by layer 'tf.compat.v1.transpose_15')>

#### dilated_bottleneck_block

In [63]:
# self.bottle = tf.keras.layers.Conv1D(filters=bottleneck,   kernel_size=1,    groups=groups)
Bottle = Conv1D(filters=Bottleneck,   kernel_size=1,    groups=groups)
Embed = Conv1D(filters=Bottleneck,   kernel_size=1,    groups=groups)(Embed)
firstlayergroups = groups
# ReLU activation function
Embed = Activation('relu')(Embed)
# Batch normalization layer
Embed = BatchNormalization(axis=-1, trainable=True, center=True, scale=True)(Embed)

'''
if firstlayergroups:
    dilated_conv = Conv1D( filters=out_channel,  kernel_size=kernel_size, dilation_rate=dilation,   padding='same',     groups=firstlayergroups  )(Embed) + tf.tile(Embed, [1,2,1])
else:
    dilated_conv = Conv1D( filters=out_channel,  kernel_size=kernel_size, dilation_rate=dilation,   padding='same',    groups=groups )(Embed)
'''

"\nif firstlayergroups:\n    dilated_conv = Conv1D( filters=out_channel,  kernel_size=kernel_size, dilation_rate=dilation,   padding='same',     groups=firstlayergroups  )(Embed) + tf.tile(Embed, [1,2,1])\nelse:\n    dilated_conv = Conv1D( filters=out_channel,  kernel_size=kernel_size, dilation_rate=dilation,   padding='same',    groups=groups )(Embed)\n"

<KerasTensor: shape=(None, 200, 32) dtype=float32 (created by layer 'tf.tile')>

In [ ]:
x = tf.repeat(x, repeats=[2], axis=-1)